In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import matplotlib.image as mpimg
#from src.utils import identify_target_variable,identify_variable_types,identify_outliers,analyze_target_balance
from ydata_profiling import ProfileReport
import glob
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.data.make_dataset import rename_columns_in_txt_files
from src.config import RAW_DATA_DIR, EXTERNAL_DATA_DIR, INTERIM_DATA_DIR



c:\Users\Administrator\Desktop\CreditRiskAnalysisProject\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-06-05 00:35:31.918 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Users\Administrator\Desktop\CreditRiskAnalysisProject


In [9]:
# Converting the raw data files by renaming their columns based on a mapping from an Excel file and saving the results in

rename_columns_in_txt_files(
    txt_folder=str(RAW_DATA_DIR),
    xls_file=str(EXTERNAL_DATA_DIR / 'PAKDD2010_VariablesList.xls'),
    output_folder=str(INTERIM_DATA_DIR)
)

3 files processed and saved to 'C:\Users\Administrator\Desktop\CreditRiskAnalysisProject\data\interim'


In [12]:
import os
print("Estás en el directorio:", os.getcwd())

Estás en el directorio: c:\Users\Administrator\Desktop\CreditRiskAnalysisProject\notebooks


In [13]:
txt_folder='../data/interim'
txt_files = glob.glob(os.path.join(txt_folder, '*.txt'))
print("Archivos encontrados:", txt_files)

import re

# Función para hacer EDA básico
def eda(df, nombre):
    print(f"\n--- EDA para {nombre} ---\n")
    
    # Mostrar primeras filas
    print("Primeras 5 filas del dataset:")
    print(df.head(5))
    
    # Identificar columnas categóricas y numéricas
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = df.select_dtypes(include=['number']).columns.tolist()
    
    print("\nColumnas categóricas:", cat_cols)
    print("Columnas numéricas:", num_cols)
    
    # Detectar outliers en columnas numéricas usando IQR
    outlier_cols = []
    for col in num_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        if not outliers.empty:
            outlier_cols.append(col)
    
    print("\nColumnas que contienen outliers:", outlier_cols if outlier_cols else "Ninguna")
    
    # Buscar columnas que contengan 'target' en el nombre (case insensitive)
    #target_cols = [col for col in df.columns if 'target' in col.lower()]
    target_cols = [col for col in df.columns if re.search('target', col, re.IGNORECASE)]

    print("\nColumnas con 'target' en el nombre:", target_cols if target_cols else "Ninguna")
    
    # Analizar balanceo si existe columna target
    if target_cols:
        for target in target_cols:
            print(f"\nAnálisis de balanceo para '{target}':")
            counts = df[target].value_counts(dropna=False)
            proportions = df[target].value_counts(normalize=True, dropna=False)
            balance_table = counts.to_frame(name='Conteo').join(proportions.to_frame(name='Proporción'))
            print(balance_table)
    else:
        print("\nNo se encontró ninguna columna target para analizar balanceo.")

# Leer y analizar cada TXT (ajusta el delimiter según corresponda)
for file in txt_files:
    try:
        df_txt = pd.read_csv(file, delimiter='\t')  # Cambia delimitador si es necesario
        eda(df_txt, file)
    except Exception as e:
        print(f"No se pudo leer {file}: {e}")


Archivos encontrados: ['../data/interim\\PAKDD2010_Leaderboard_Data_with_columns.txt', '../data/interim\\PAKDD2010_Modeling_Data_with_columns.txt', '../data/interim\\PAKDD2010_Prediction_Data_with_columns.txt']

--- EDA para ../data/interim\PAKDD2010_Leaderboard_Data_with_columns.txt ---

Primeras 5 filas del dataset:
   ID_CLIENT CLERK_TYPE  PAYMENT_DAY APPLICATION_SUBMISSION_TYPE  \
0      50001          C            1                         Web   
1      50002          C           20                         Web   
2      50003          C           25                         Web   
3      50004          C           10                         Web   
4      50005          C           25                         Web   

   QUANT_ADDITIONAL_CARDS  POSTAL_ADDRESS_TYPE SEX  MARITAL_STATUS  \
0                       0                    1   F               1   
1                       0                    1   F               1   
2                       0                    1   M           

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2960\584321934.py:56: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_txt = pd.read_csv(file, delimiter='\t')  # Cambia delimitador si es necesario



--- EDA para ../data/interim\PAKDD2010_Modeling_Data_with_columns.txt ---

Primeras 5 filas del dataset:
   ID_CLIENT CLERK_TYPE  PAYMENT_DAY APPLICATION_SUBMISSION_TYPE  \
0          1          C            5                         Web   
1          2          C           15                       Carga   
2          3          C            5                         Web   
3          4          C           20                         Web   
4          5          C           10                         Web   

   QUANT_ADDITIONAL_CARDS  POSTAL_ADDRESS_TYPE SEX  MARITAL_STATUS  \
0                       0                    1   F               6   
1                       0                    1   F               2   
2                       0                    1   F               2   
3                       0                    1   F               2   
4                       0                    1   M               2   

   QUANT_DEPENDANTS  EDUCATION_LEVEL  ... FLAG_HOME_ADDRESS_DOCU